In [1]:
import pandas as pd
import itertools

# Define variables
mn = 75
adj_mn = mn * 0.8
mx = 862
adj_mx = mx * 1.35
mx1 = mx * .5
strike_num = 4
diff = mx - mn
a = diff / strike_num
b = a*0.7
c = b*0.7

# Create X1 DataFrame
X1 = pd.DataFrame({'X1': range(int(mn), int(adj_mx), 5)})




In [2]:
adj_mn

60.0

In [3]:
mx1

431.0

In [4]:
X1

,X1
0,75
1,80
2,85
3,90
4,95
...,...
213,1140
214,1145
215,1150
216,1155


In [ ]:
def generate_strike_combinations(X1):
    for x1, x2, x3, x4 in itertools.product(X1['X1'], repeat=strike_num):
        if (x1 < x2 < x3 < x4) and (x1 < mx1) and (x4 > mx1) and  ((x3 - x2) >= (x2 - x1)*0.99) and ((x4 - x3) >= (x3 - x2)*0.99) and ((x2 - x1) >= (x3 - x2) >= (x4 - x3)):
            yield {'X1': x1, 'X2': x2, 'X3': x3, 'X4': x4}

# Create strike DataFrame using generator
strike = pd.DataFrame(generate_strike_combinations(X1))

In [ ]:
strike

In [ ]:
df=pd.read_excel('Sample_index_data.xlsx',sheet_name='final')

In [ ]:
df.columns

In [ ]:
df=df.sort_values(['District', 'ID', 'Year'])

In [ ]:
df['District'].unique()

In [ ]:
wt=pd.read_excel('Sample_index_data.xlsx',sheet_name='wt')

In [ ]:
wt['Geotag Area'].sum()

In [ ]:
wt

In [ ]:
wt['wt']=wt['Geotag Area']/4890

In [ ]:
wt=wt[['District', 'ID','wt']]

In [ ]:
df.columns

In [ ]:
wt.columns

In [ ]:
strike['District']='HIMACHAL PRADESH_Kangra'

In [ ]:
df.columns

In [ ]:
import numpy as np
final_data = []

# Iterate over each row in the second dataframe
for index, row in strike.iterrows():
    # Extract 'ID', 'X1', 'X2', 'X3', values for each row
    District = row['District']
    x1 = row['X1']
    x2 = row['X2']
    x3 = row['X3']
    x4 = row['X4']
    
    # Filter the rows from the first dataframe based on the 'District'
    filtered_rows = df[df['District'] == District].copy()  # Copy the filtered rows
    
    # Apply the logic to calculate 'Burn cost' for the current row
    filtered_rows['Brn'] = np.where(filtered_rows['Index'] < x1, 0, np.nan)
    filtered_rows['Brn'] = np.where((filtered_rows['Index'] >= x1) & (filtered_rows['Index'] < x2), 10, filtered_rows['Brn'])
    filtered_rows['Brn'] = np.where((filtered_rows['Index'] >= x2) & (filtered_rows['Index'] < x3), 25, filtered_rows['Brn'])
    filtered_rows['Brn'] = np.where((filtered_rows['Index'] >= x3) & (filtered_rows['Index'] < x4), 60, filtered_rows['Brn'])
    filtered_rows['Brn'] = np.where(filtered_rows['Index'] >= x4, 100, filtered_rows['Brn'])
    
    values_to_count = [10, 25, 60, 100]
    results = pd.DataFrame()
    for value in values_to_count:
        value_counts = filtered_rows[filtered_rows['Brn'] == value].groupby(['District']).size().reset_index(name=f'Count_{value}')
        if results.empty:
            results = value_counts
        else:
            results = results.merge(value_counts, on=['District'], how='outer')
            
    results = results.fillna(0)
    
    # Ensure all Count columns exist
    for col in ['Count_10', 'Count_25', 'Count_60', 'Count_100']:
        if col not in results.columns:
            results[col] = 0
    
    results['tot'] = results['Count_10'] + results['Count_25'] + results['Count_60'] + results['Count_100']
    id_cnt = filtered_rows.groupby('District')['ID'].nunique().reset_index()
    id_cnt.columns = ['District', 'grd_cnt']
    results = pd.merge(results, id_cnt)
    results['freq'] = results['tot'] / results['grd_cnt']
    
    # Calculate the mean of 'Brn' for different time periods (last 10, 20, 30 rows)
    avg10 = filtered_rows.groupby(['District', 'ID']).tail(10).groupby(['District', 'ID'])['Brn'].mean().reset_index()
    avg20 = filtered_rows.groupby(['District', 'ID']).tail(20).groupby(['District', 'ID'])['Brn'].mean().reset_index()
    avg30 = filtered_rows.groupby(['District', 'ID']).tail(30).groupby(['District', 'ID'])['Brn'].mean().reset_index()
                                    
    z = pd.concat([avg10, avg20, avg30])
    z = z.sort_values(['District', 'ID'])
    z1 = z.groupby(['District', 'ID']).mean().reset_index()
    z2 = pd.merge(z1, wt)
    z2['weigh_avg'] = z2['Brn'] * z2['wt']
    wAvg = z2.groupby(['District'])['weigh_avg'].sum()
    wAvg = wAvg.iloc[0]
    
    # Append 'District', 'X1', 'X2', 'X3', 'X4', 'wAvg', and results columns to the final_data list for each row
    final_data.append([
        District, 
        x1, 
        x2, 
        x3, 
        x4, 
        wAvg, 
        results.iloc[0]['Count_10'], 
        results.iloc[0]['Count_25'], 
        results.iloc[0]['Count_60'], 
        results.iloc[0]['Count_100'], 
        results.iloc[0]['tot'], 
        results.iloc[0]['grd_cnt'], 
        results.iloc[0]['freq']
    ])

# Create a DataFrame from the final_data list with appropriate column names
final_df = pd.DataFrame(final_data, columns=[
    'District', 'X1', 'X2', 'X3', 'X4', 'Brn', 'Count_10', 'Count_25', 'Count_60', 'Count_100', 'tot', 'grd_cnt', 'freq'
])

# Display or return the resulting DataFrame with 'District', 'X1', 'X2', 'X3', 'X4', 'Brn', and results values for each row
print(final_df)

In [ ]:
final_df

In [ ]:
b=final_df[final_df['Brn'].between(2.75,3.5)]

In [ ]:
b=b[b['freq'].between(5.5,7)]

In [ ]:
b

In [ ]:
a=final_df[final_df['Brn'].between(2.75,3.5)]
a=a[a['freq'].between(5.5,7.5)]

In [ ]:
#a=a[a['Count_60']==1]

In [ ]:
a

In [ ]:
a=a[a['freq'].between(5.5,7)]

In [ ]:
a

In [ ]:
a.to_excel('sample_op.xlsx')

In [ ]:
import pandas as pd
import numpy as np

# Assuming 'df' is the first dataframe and 'strike' is the second dataframe

# Create an empty list to store the results
final_data = []

# Iterate over each row in the second dataframe
for index, row in strike.iterrows():
    # Extract 'ID', 'X1', 'X2', 'X3', values for each row
    District = row['District']
    x1 = row['X1']
    x2 = row['X2']
    x3 = row['X3']
    x4 = row['X4']
    
    # Filter the rows from the first dataframe based on the 'ID'
    filtered_rows = df[df['District'] == District].copy()  # Copy the filtered rows
    
    # Apply the logic to calculate 'Brn' for the current row
    filtered_rows['Brn'] = np.where(filtered_rows['Index'] < x1, 0, np.nan)
    filtered_rows['Brn'] = np.where((filtered_rows['Index'] >= x1) & (filtered_rows['Index'] < x2), 10, filtered_rows['Brn'])
    filtered_rows['Brn'] = np.where((filtered_rows['Index'] >= x2) & (filtered_rows['Index'] < x3), 25, filtered_rows['Brn'])
    filtered_rows['Brn'] = np.where((filtered_rows['Index'] >= x3) & (filtered_rows['Index'] < x4), 60, filtered_rows['Brn'])
    filtered_rows['Brn'] = np.where(filtered_rows['Index'] >= x4, 100, filtered_rows['Brn'])
    
    values_to_count = [10, 25, 60, 100]
    results = pd.DataFrame()
    for value in values_to_count:
        value_counts = filtered_rows[filtered_rows['Brn'] == value].groupby(['District']).size().reset_index(name=f'Count_{value}')
        if results.empty:
            results = value_counts
        else:
            results = results.merge(value_counts, on=['District'], how='outer')
            
    results = results.fillna(0)
    results['tot'] = results['Count_10'] + results['Count_25'] + results['Count_60'] + results['Count_100']
    id_cnt = filtered_rows.groupby('District')['ID'].nunique().reset_index()
    id_cnt.columns = ['District', 'grd_cnt']
    results = pd.merge(results, id_cnt)
    results['freq'] = results['tot'] / results['grd_cnt']
    
    # Calculate the mean of 'Brn' for different time periods (last 10, 20, 30 rows)
    avg10 = filtered_rows.groupby(['District', 'ID']).tail(10).groupby(['District', 'ID'])['Brn'].mean().reset_index()
    avg20 = filtered_rows.groupby(['District', 'ID']).tail(20).groupby(['District', 'ID'])['Brn'].mean().reset_index()
    avg30 = filtered_rows.groupby(['District', 'ID']).tail(30).groupby(['District', 'ID'])['Brn'].mean().reset_index()
                                    
    z = pd.concat([avg10, avg20, avg30])
    z = z.sort_values(['District', 'ID'])
    z1 = z.groupby(['District', 'ID']).mean().reset_index()
    z2 = pd.merge(z1, wt)
    z2['weigh_avg'] = z2['Brn'] * z2['wt']
    wAvg = z2.groupby(['District'])['weigh_avg'].sum()
    wAvg = wAvg.iloc[0]
    
    # Append 'District', 'X1', 'X2', 'X3', 'X4', 'wAvg', and results columns to the final_data list for each row
    final_data.append([
        District, 
        x1, 
        x2, 
        x3, 
        x4, 
        wAvg, 
        results.iloc[0]['Count_10'], 
        results.iloc[0]['Count_25'], 
        results.iloc[0]['Count_60'], 
        results.iloc[0]['Count_100'], 
        results.iloc[0]['tot'], 
        results.iloc[0]['grd_cnt'], 
        results.iloc[0]['freq']
    ])

# Create a DataFrame from the final_data list with appropriate column names
final_df = pd.DataFrame(final_data, columns=[
    'District', 'X1', 'X2', 'X3', 'X4', 'Brn', 'Count_10', 'Count_25', 'Count_60', 'Count_100', 'tot', 'grd_cnt', 'freq'
])

# Display or return the resulting DataFrame with 'District', 'X1', 'X2', 'X3', 'X4', 'Brn', and results values for each row
print(final_df)


In [ ]:
final_df.head(5)

In [ ]:
a=final_df[final_df['Brn'].between(2.75,3.25)]

In [ ]:
a.to_excel('Himachal_op.xlsx')

In [ ]:
a.shape

In [ ]:
filtered_rows=df

In [ ]:
    filtered_rows['Brn'] = np.where(filtered_rows['Index'] < 110, 0, np.nan)
    filtered_rows['Brn'] = np.where((filtered_rows['Index'] >= 110) & (filtered_rows['Index'] < 210), 10, filtered_rows['Brn'])
    filtered_rows['Brn'] = np.where((filtered_rows['Index'] >= 210) & (filtered_rows['Index'] < 310), 25, filtered_rows['Brn'])
    filtered_rows['Brn'] = np.where((filtered_rows['Index'] >= 310) & (filtered_rows['Index'] < 410), 60, filtered_rows['Brn'])
    filtered_rows['Brn'] = np.where(filtered_rows['Index'] >= 410, 100, filtered_rows['Brn'])

In [ ]:
filtered_rows.sort_values(['Index'])

In [ ]:
    values_to_count = [10, 25, 60, 100]
    results = pd.DataFrame()
    for value in values_to_count:
        value_counts = filtered_rows[filtered_rows['Brn'] == value].groupby(['District']).size().reset_index(name=f'Count_{value}')
        if results.empty:
            results = value_counts
        else:
            results = results.merge(value_counts, on=['District'], how='outer')
            
    results = results.fillna(0)
    results['tot'] = results['Count_10'] + results['Count_25'] + results['Count_60'] + results['Count_100']
    id_cnt = filtered_rows.groupby('District')['ID'].nunique().reset_index()
    id_cnt.columns = ['District', 'grd_cnt']
    results = pd.merge(results, id_cnt)
    results['freq'] = results['tot'] / results['grd_cnt']

In [ ]:
results

In [ ]:
    avg10 = filtered_rows.groupby(['District', 'ID']).tail(10).groupby(['District', 'ID'])['Brn'].mean().reset_index()
    avg20 = filtered_rows.groupby(['District', 'ID']).tail(20).groupby(['District', 'ID'])['Brn'].mean().reset_index()
    avg30 = filtered_rows.groupby(['District', 'ID']).tail(30).groupby(['District', 'ID'])['Brn'].mean().reset_index()
                                    
    z = pd.concat([avg10, avg20, avg30])
    z = z.sort_values(['District', 'ID'])
    z1 = z.groupby(['District', 'ID']).mean().reset_index()
    z2 = pd.merge(z1, wt)
    z2['weigh_avg'] = z2['Brn'] * z2['wt']
    wAvg = z2.groupby(['District'])['weigh_avg'].sum()
    wAvg = wAvg.iloc[0]

In [ ]:
    final_data.append([
        District, 
        x1, 
        x2, 
        x3, 
        x4, 
        wAvg, 
        results.iloc[0]['Count_10'], 
        results.iloc[0]['Count_25'], 
        results.iloc[0]['Count_60'], 
        results.iloc[0]['Count_100'], 
        results.iloc[0]['tot'], 
        results.iloc[0]['grd_cnt'], 
        results.iloc[0]['freq']
    ])

In [ ]:
results.iloc[0]['Count_10']

In [ ]:
filtered_rows=df

In [ ]:
    filtered_rows['Brn'] = np.where(filtered_rows['Index'] < x1, 0, np.nan)
    filtered_rows['Brn'] = np.where((filtered_rows['Index'] >= x1) & (filtered_rows['Index'] < x2), 10, filtered_rows['Brn'])
    filtered_rows['Brn'] = np.where((filtered_rows['Index'] >= x2) & (filtered_rows['Index'] < x3), 25, filtered_rows['Brn'])
    filtered_rows['Brn'] = np.where((filtered_rows['Index'] >= x3) & (filtered_rows['Index'] < x4), 60, filtered_rows['Brn'])
    filtered_rows['Brn'] = np.where(filtered_rows['Index'] >= x4, 100, filtered_rows['Brn'])
    
    values_to_count = [10, 25, 60, 100]
    results = pd.DataFrame()
    for value in values_to_count:
        value_counts = filtered_rows[filtered_rows['Brn'] == value].groupby(['District']).size().reset_index(name=f'Count_{value}')
        if results.empty:
            results = value_counts
        else:
            results = results.merge(value_counts, on=['District'], how='outer')
            
    results = results.fillna(0)
    
    # Ensure all Count columns exist
    for col in ['Count_10', 'Count_25', 'Count_60', 'Count_100']:
        if col not in results.columns:
            results[col] = 0
    
    results['tot'] = results['Count_10'] + results['Count_25'] + results['Count_60'] + results['Count_100']
    id_cnt = filtered_rows.groupby('District')['ID'].nunique().reset_index()
    id_cnt.columns = ['District', 'grd_cnt']
    results = pd.merge(results, id_cnt)
    results['freq'] = results['tot'] / results['grd_cnt']
    
    # Calculate the mean of 'Brn' for different time periods (last 10, 20, 30 rows)
    avg10 = filtered_rows.groupby(['District', 'ID']).tail(10).groupby(['District', 'ID'])['Brn'].mean().reset_index()
    avg20 = filtered_rows.groupby(['District', 'ID']).tail(20).groupby(['District', 'ID'])['Brn'].mean().reset_index()
    avg30 = filtered_rows.groupby(['District', 'ID']).tail(30).groupby(['District', 'ID'])['Brn'].mean().reset_index()
                                    
    z = pd.concat([avg10, avg20, avg30])
    z = z.sort_values(['District', 'ID'])
    z1 = z.groupby(['District', 'ID']).mean().reset_index()
    z2 = pd.merge(z1, wt)
    z2['weigh_avg'] = z2['Brn'] * z2['wt']
    wAvg = z2.groupby(['District'])['weigh_avg'].sum()
    wAvg = wAvg.iloc[0]